In [7]:
# load env ------------------------------------------------------------------------
import os
import utils

utils.load_env()
os.environ['LANGCHAIN_TRACING_V2'] = "false"


# debug ------------------------------------------------------------------
from langchain.globals import set_debug, set_verbose
set_verbose(True)
set_debug(False)

from langchain_core.messages import (
    AIMessage, 
    HumanMessage,
    ToolMessage
)
from langgraph.graph import END, StateGraph, START
from agents import(
    AgentState,
    agents_metadata,
    agent_names
)
from tools import get_tools_output, all_tools
from chat_history import save_chat_history, load_chat_history
from langgraph.checkpoint.memory import MemorySaver

## Define Tool Node
from langgraph.prebuilt import ToolNode
from typing import Literal

tool_node = ToolNode(all_tools)

def router(state) -> Literal["call_tool", "continue", "supervisor", "creative_communication_agent", "crm_agent", "__end__"]:
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if "FINALANSWER" in last_message.content:
        return "__end__"
    if "supervisor" in last_message.content:
        return "supervisor"
    if "creative_communication_agent" in last_message.content:
        return "creative_communication_agent"
    if "crm_agent" in last_message.content:
        return "crm_agent"
    if last_message.tool_calls:
        # The previous agent is invoking a tool
        return "call_tool"
    else:
        return "continue"


## Workflow Graph ------------------------------------------------------------------------
workflow = StateGraph(AgentState)

# add agent nodes
for name, value in agents_metadata.items():
    workflow.add_node(name, value['node'])
    
workflow.add_node("call_tool", tool_node)

workflow.add_conditional_edges(
    "supervisor",
    router,
    {
        "crm_agent":"crm_agent",
        "creative_communication_agent":"creative_communication_agent",
        "call_tool": "call_tool",
        "__end__": END,
        "continue": END, 
        }
)

workflow.add_conditional_edges(
    "creative_communication_agent",
    router,
    {
        "call_tool": "call_tool",
        "continue": "supervisor", 
        }
)

workflow.add_conditional_edges(
    "crm_agent",
    router,
    {
        "call_tool": "call_tool",
        "continue": "supervisor", 
        }
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {name:name for name in agent_names},
)

workflow.add_edge(START, "supervisor")
graph = workflow.compile()

def submitUserMessage(
    user_input:str, 
    user_id:str="test", 
    keep_chat_history:bool=False, 
    return_reference:bool=False, 
    verbose:bool=False,
    recursion_limit:int=20
    ) -> str:
    
    os.environ['CURRENT_USER_ID'] = user_id
    
    chat_history = load_chat_history(user_id=user_id) if keep_chat_history else []
    chat_history = chat_history[-20:]
    
    # memory only keep chat history only along agents.
    internal_level_memory = MemorySaver()
    graph = workflow.compile(checkpointer=internal_level_memory)

    events = graph.stream(
        {
            "messages": [
                HumanMessage(
                    user_input
                )
            ],
            "chat_history": chat_history
        },
        # Maximum number of steps to take in the graph
        {"recursion_limit": recursion_limit, "thread_id":"a"},
    )
    
    if not verbose:
        events = [e for e in events]
        response = list(events[-1].values())[0]
    else:
        for e in events:
            a = list(e.items())[0]
            a[1]['messages'][0].pretty_print()
        
        response = a[1]
    
    response = response["messages"][0].content
    response = utils.format_bot_response(response, markdown=True)
    
    if keep_chat_history:
        save_chat_history(bot_message=response, human_message=user_input, user_id=user_id)
    
    if return_reference:
        return response, get_tools_output()
    else:
        return response

In [8]:
# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
# except Exception:
#     # This requires some extra dependencies and is optional
#     pass

In [9]:
# import utils
result = submitUserMessage("ฉันชอบ iphone", keep_chat_history=True, return_reference=True, verbose=True)
utils.notify(sound_effect="purr")

================================== Ai Message ==================================
Name: supervisor

ฉันจะส่งข้อมูลนี้ไปยัง CRM Agent เพื่อดึงข้อมูลลูกค้าเพิ่มเติมเกี่ยวกับความชอบของคุณค่ะ

ส่งข้อความไปยัง CRM Agent:

"สวัสดีค่ะ CRM Agent, 
ลูกค้ารายนี้ได้แจ้งว่าพวกเขาชอบ iPhone กรุณาช่วยดึงข้อมูลเพิ่มเติมเกี่ยวกับลูกค้ารายนี้เพื่อให้เราสามารถให้บริการที่ตรงตามความต้องการของพวกเขาได้ค่ะ ขอบคุณค่ะ!"


In [10]:
submitUserMessage("ช่่วยแนะนำสินค้าหน่่อย", keep_chat_history=True, return_reference=True, verbose=True)

================================== Ai Message ==================================
Name: supervisor

ตอนนี้ฉันจะส่งข้อมูลนี้ไปยัง Creative Communication Agent เพื่อให้พวกเขาช่วยในการแนะนำสินค้าที่เกี่ยวข้องกับ iPhone ค่ะ

ส่งข้อความไปยัง Creative Communication Agent:

"สวัสดีค่ะ Creative Communication Agent,
ลูกค้ารายนี้ได้แจ้งว่าพวกเขาชอบ iPhone และขอให้เราช่วยแนะนำสินค้าที่เกี่ยวข้อง กรุณาช่วยจัดทำคำแนะนำสินค้าที่เหมาะสมกับความชอบของลูกค้าได้ไหมคะ ขอบคุณค่ะ!"


('ตอนนี้ฉันจะส่งข้อมูลนี้ไปยัง Creative Communication Agent เพื่อให้พวกเขาช่วยในการแนะนำสินค้าที่เกี่ยวข้องกับ iPhone ค่ะ\n\nส่งข้อความไปยัง Creative Communication Agent:\n\n"สวัสดีค่ะ Creative Communication Agent,\nลูกค้ารายนี้ได้แจ้งว่าพวกเขาชอบ iPhone และขอให้เราช่วยแนะนำสินค้าที่เกี่ยวข้อง กรุณาช่วยจัดทำคำแนะนำสินค้าที่เหมาะสมกับความชอบของลูกค้าได้ไหมคะ ขอบคุณค่ะ!"',
 '')